마르코프 체인은 확률을 기반으로 문장을 이어 붙여 나가는 방법이며, LSTM/RNN 은 머신러닝으로 다음에 위치할 문장을 예측해서 문장을 생성하는 방법입니다.

### 마르코프 체인이란?

현재 상타만을 기반으로 다음 상태를 선택



In [7]:
import codecs
from bs4 import BeautifulSoup
from konlpy.tag import Twitter
import urllib.request

import os,re,json,random



In [11]:
dirname = '../data/text/토지/'
files = os.listdir(dirname)

text = ""
for filename in files:
    if filename.startswith('토지'):
        fullname = os.path.join(dirname, filename)
        fp = codecs.open(fullname, 'r', encoding='cp949', errors='ignore')
        text += fp.read()
print(len(text))

toji = os.path.join(dirname, 'toji.txt')
f = open(toji, 'w', encoding='utf-16')
f.write(text)
f.close()


5382203


In [12]:
fp = codecs.open(toji, 'r', encoding='utf-16')

In [13]:
text = fp.read()

text = text.replace('...', '')

In [17]:
# 문장에서 3 개의 연속된 단어를 가지고 이들의 조합을 key 로하고 빈도수를 value로 하는 dict 를 만든다.

def make_dic(words):
    tmp = ['@']
    dic = {}
    for word in words:
        tmp.append(word)
        if len(tmp) < 3: continue
        if len(tmp) > 3: tmp = tmp[1:]
        set_word3(dic, tmp)
        if word == '.':
            tmp = ['@']
            continue
    return dic

def set_word3(dic, s3):
    w1, w2, w3 = s3
    if not w1 in dic: dic[w1] = {}
    if not w2 in dic[w1]: dic[w1][w2] = {}
    if not w3 in dic[w1][w2]: dic[w1][w2][w3] = 0
    dic[w1][w2][w3] += 1
    
def make_sentence(dic):
    ret = []
    if not '@' in dic: return 'no dic'
    top = dic['@']
    w1 = word_choice(top)
    w2 = word_choice(w1)
    ret.append(w1)
    ret.append(w2)
    while True:
        w3 = word_choice(dic[w1][w2])
        ret.append(w3)
        if w3 == '.' : break
        w1, w2 = w2, w3
    ret = ''.join(ret)
    
    # 띄어쓰기
    params = urllib.parse.urlencode({
        "_callback":"",
        "q": ret
    })
    
    # 네이버 맞춤법 검사기
    data = urllib.request.urlopen("https://m.search.naver.com/p/csearch/dcontent/spellchecker.nhn?" + params)
    data = data.read().decode('utf-8')[1:-2]
    # '({"message":{"@type":"response","@service":"DBSearchSpellchecker","
    #@version":"1.1.0","result":{ "errata_count":0,"html":"아버지 가방에 들어가신다."}}});'
    # 앞의 ( 와 뒤의  ); 제거
    
    data = json.loads(data)
    data = data['message']['result']['html']
    ddata = soup = BeautifulSoup(data, 'html.parser').getText()
    return data
    
def word_choice(sel):
    keys = sel.keys()
    return random.choice(list(keys))

    
            


In [18]:
twitter = Twitter()
mallist = twitter.pos(text, norm=True)
words = []

dict_file = os.path.join(dirname, 'markov-toji.json')
for word in mallist:
    if not word[1] in ['Punctuation']:
        words.append(word[0])
    if word[0] == '.':
        words.append(word[0])
    
    dic = make_dic(words)
    json.dump(dic, open(dict_file, 'w', encoding='utf-8'))
    

UnboundLocalError: local variable 'temp' referenced before assignment

In [ ]:
dic = json.load(open(dic_file, 'r'))

for i in range(3):
    s = make_sentence(dic)
    print(s)
    print('---')

In [27]:
def naver(text):
    params = urllib.parse.urlencode({
        "_callback":"",
        "q": text
    })
    data = urllib.request.urlopen("https://m.search.naver.com/p/csearch/dcontent/spellchecker.nhn?" + params)
    data = data.read().decode('utf-8')[1:-2]
    data = json.loads(data)
    data = data['message']['result']['html']
    ddata = soup = BeautifulSoup(data, 'html.parser').getText()
    return data

In [30]:
naver('그러면 않되나요?')

"그러면 <span class='re_red'>안 되나요?</span>"

### LSTM/RNN

LSTM (Log Short Term-Memory)

RNN (Recurrent Neural Network)

